# dbt & AWS Glue demo

## Project Run

### Load csv files

In [6]:
!dbt seed --profiles-dir ../config/ --select raw_customers

16:19:16  Running with dbt=1.3.1
16:19:16  Found 5 models, 20 tests, 1 snapshot, 0 analyses, 363 macros, 0 operations, 3 seed files, 3 sources, 1 exposure, 0 metrics
16:19:16  
16:20:13  Concurrency: 1 threads (target='dev')
16:20:13  
16:20:13  1 of 1 START seed file jaffle_db.raw_customers ................................. [RUN]
16:20:47  1 of 1 OK loaded seed file jaffle_db.raw_customers ............................. [CREATE 102 in 34.34s]
16:20:49  
16:20:49  Finished running 1 seed in 0 hours 1 minutes and 32.55 seconds (92.55s).
16:20:49  
16:20:49  Completed successfully
16:20:49  
16:20:49  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1
sys:1: ResourceWarning: unclosed <ssl.SSLSocket fd=10, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.49.210.49', 50721), raddr=('54.171.112.47', 443)>


### Execute project

In [7]:
! dbt run --profiles-dir ../config/ --select customers

16:21:44  Running with dbt=1.3.1
16:21:45  Found 5 models, 20 tests, 1 snapshot, 0 analyses, 363 macros, 0 operations, 3 seed files, 3 sources, 1 exposure, 0 metrics
16:21:45  
16:22:29  Concurrency: 1 threads (target='dev')
16:22:29  
16:22:29  1 of 1 START sql incremental model jaffle_db.customers ......................... [RUN]
16:23:56  1 of 1 OK created sql incremental model jaffle_db.customers .................... [OK in 87.84s]
16:23:58  
16:23:58  Finished running 1 incremental model in 0 hours 2 minutes and 13.73 seconds (133.73s).
16:23:59  
16:23:59  Completed successfully
16:23:59  
16:23:59  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 TOTAL=1
sys:1: ResourceWarning: unclosed <ssl.SSLSocket fd=10, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.49.210.49', 50818), raddr=('54.77.108.156', 443)>


### Perform Test

In [22]:
! dbt test --profiles-dir ../config/

16:32:21  Running with dbt=1.3.1
16:32:21  Found 5 models, 20 tests, 1 snapshot, 0 analyses, 363 macros, 0 operations, 3 seed files, 3 sources, 1 exposure, 0 metrics
16:32:21  
^C


### Perform Snapshot

In [ ]:
! dbt snapshot --profiles-dir ../config/

### Generate the documentation

In [ ]:
! dbt docs generate --profiles-dir ../config/

### Expose the documentation

In [ ]:
! dbt docs serve --profiles-dir ../config/

### Incremental test

In [31]:
!dbt seed --profiles-dir ../config/

16:52:47  Running with dbt=1.3.1
16:52:47  Unable to do partial parsing because a project config has changed
16:52:47  Found 5 models, 20 tests, 1 snapshot, 0 analyses, 363 macros, 0 operations, 3 seed files, 3 sources, 1 exposure, 0 metrics
16:52:47  
16:54:08  Concurrency: 1 threads (target='dev')
16:54:08  
16:54:08  1 of 3 START seed file jaffle_db.raw_customers ................................. [RUN]
16:54:39  1 of 3 OK loaded seed file jaffle_db.raw_customers ............................. [CREATE 101 in 31.10s]
16:54:39  2 of 3 START seed file jaffle_db.raw_orders .................................... [RUN]
16:54:42  2 of 3 OK loaded seed file jaffle_db.raw_orders ................................ [CREATE 103 in 3.64s]
16:54:42  3 of 3 START seed file jaffle_db.raw_payments .................................. [RUN]
16:54:47  3 of 3 OK loaded seed file jaffle_db.raw_payments .............................. [CREATE 117 in 4.83s]
16:54:48  
16:54:48  Finished running 3 seeds in 0 hours 

In [32]:
! dbt run --profiles-dir ../config/ --select customers orders

16:54:51  Running with dbt=1.3.1
16:54:51  Found 5 models, 20 tests, 1 snapshot, 0 analyses, 363 macros, 0 operations, 3 seed files, 3 sources, 1 exposure, 0 metrics
16:54:51  
16:56:03  Concurrency: 1 threads (target='dev')
16:56:03  
16:56:03  1 of 2 START sql incremental model jaffle_db.customers ......................... [RUN]
16:57:27  1 of 2 OK created sql incremental model jaffle_db.customers .................... [OK in 84.34s]
16:57:27  2 of 2 START sql incremental model jaffle_db.orders ............................ [RUN]
16:58:19  2 of 2 OK created sql incremental model jaffle_db.orders ....................... [OK in 51.77s]
16:58:21  
16:58:21  Finished running 2 incremental models in 0 hours 3 minutes and 30.08 seconds (210.08s).
16:58:21  
16:58:21  Completed successfully
16:58:21  
16:58:21  Done. PASS=2 WARN=0 ERROR=0 SKIP=0 TOTAL=2
sys:1: ResourceWarning: unclosed <ssl.SSLSocket fd=10, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.8.

## Clean Demo

In [24]:
import boto3

In [26]:
bucket_name = "demo-dbt-output-788996449432"

s3_client = boto3.client("s3")

response = s3_client.list_objects_v2(Bucket=bucket_name)

files_in_folder = response["Contents"]
files_to_delete = []

for f in files_in_folder:
    files_to_delete.append({"Key": f["Key"]})

response = s3_client.delete_objects(
    Bucket=bucket_name, Delete={"Objects": files_to_delete}
)

In [27]:
client = boto3.client('glue')
response = client.delete_database(Name='jaffle_db')